In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna, joblib, matplotlib, os, pyfolio, sys
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from pyfolio import timeseries

matplotlib.use('Agg')
%matplotlib inline
import datetime
from stockstats import StockDataFrame as Sdf
from QuantumAI.config import config
from QuantumAI.config import config_tickers
from QuantumAI.preprocessors.yahoodownloader import YahooDownloader
from QuantumAI.preprocessors.preprocessors import FeatureEngineer, data_split
# from QuantumAI.env.env_portfolio import StockPortfolioEnv
# from QuantumAI.env.env_portfolioSPX import StockPortfolioEnv
from QuantumAI.env.env_portfolio_original import StockPortfolioEnv
from QuantumAI.env.env_portfolioSPXvsCash import SPXvsCASH
from QuantumAI.models.models import DRLAgent
from QuantumAI.plot.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts, show_weight_distribution
from QuantumAI.preprocessors.data_processor import DataProcessor
from QuantumAI.preprocessors.processor_yahoofinance import YahooFinanceProcessor
from QuantumAI.hyper_opt.hyperopt1 import optimize, optimize2

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
bbg_full_start_date="1991-01-02"
full_end_date="2023-07-25"
bbg_train_start_date="1991-01-02"
bbg_train_end_date="2015-12-31"
bbg_validate_start_date="2016-01-01"
bbg_validate_end_date="2020-12-31"
bbg_test_start_date="2021-01-01"
bbg_test_end_date="2023-07-25"
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [3]:
df =  pd.read_pickle("pickle.pkl")

In [4]:
train = data_split(df, bbg_train_start_date, bbg_train_end_date)
validate = data_split(df, bbg_validate_start_date, bbg_validate_end_date)
test = data_split(df, bbg_test_start_date, bbg_test_end_date)

In [5]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 2, State Space: 2


In [6]:
all_techs = ['RSI14', 'RSI30', 'RSI3',
       'MA200', 'MA50', 'MA20', 'PUT Index', 'RXM Index',
       'USGG2YR Index', 'USGG10YR Index', 'MOVE Index', 'VIX Index',
       'NWHLSENY Index', 'NWHLSEND Index', 'USGG5YR Index', 'SFFRNEWS Index',
       'US10-US2Y', 'macd', 'boll_ub', 'boll_lb',  'cci_30', 'dx_30',
       'close_30_sma', 'close_60_sma']
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension,
    # "eco_indicator_list": eco_indicator_list, 
    # "tech_indicator_list": new_tech_indics, 
    "tech_indicator_list": all_techs, 
    # "tech_indicator_list": config.INDICATORS, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    # "cov": False
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
# !FLO New validation set
e_validate_gym = StockPortfolioEnv(df = validate, **env_kwargs)

Observation space Box(-inf, inf, (26, 2), float32)
state space 2
Observation space Box(-inf, inf, (26, 2), float32)
state space 2


In [7]:
env_train, _ = e_train_gym.get_sb_env()
env_validate, _ = e_validate_gym.get_sb_env()

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning:

You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.



In [8]:
optimize2(e_validate_gym=e_validate_gym, env_train=e_train_gym,n_trial_runs=10)

[I 2023-07-27 18:18:32,643] A new study created in memory with name: no-name-2a3856e3-b60e-49bd-9bbf-ca10cc1e5fb0
/Users/floriankockler/Documents/GitHub.nosync/QuantumAI/QuantumAI/hyper_opt/hyperopt1.py:77: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



begin_total_asset:1000000
end_total_asset:14922377.817873197
Sharpe:  0.5979031725851943
begin_total_asset:1000000
end_total_asset:13948074.887828235
Sharpe:  0.5856783779035942
begin_total_asset:1000000
end_total_asset:12913248.273210445
Sharpe:  0.5740060972368966
begin_total_asset:1000000
end_total_asset:14922112.297020774
Sharpe:  0.5966636291734375
begin_total_asset:1000000
end_total_asset:12678535.87027149
Sharpe:  0.5686541465202739
begin_total_asset:1000000
end_total_asset:11446442.169515137
Sharpe:  0.5504884284171644
begin_total_asset:1000000
end_total_asset:13991785.96415106
Sharpe:  0.5889517771933533
begin_total_asset:1000000
end_total_asset:14144980.639753096
Sharpe:  0.5889307214660204
begin_total_asset:1000000
end_total_asset:13736815.787540033
Sharpe:  0.5844026728722068
begin_total_asset:1000000
end_total_asset:16217699.574934283
Sharpe:  0.6132709758833329
begin_total_asset:1000000
end_total_asset:11630415.836328465
Sharpe:  0.5542241633654298
begin_total_asset:10000

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning:

Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.



begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003
begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003
begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003
begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003
begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003
begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003
begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003
begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003
begin_total_asset:1000000
end_total_asset:2309149.2073343312
Sharpe:  0.9266266023272003


[I 2023-07-27 18:19:56,662] Trial 0 finished with value: 1818342038.625 and parameters: {'learning_rate': 3.2712953575267733e-05, 'n_steps': 11, 'gamma': 0.9606195153590082, 'gae_lambda': 0.9373956991640547, 'ent_coef': 0.024340126280744393, 'vf_coef': 0.38831768057115196, 'max_grad_norm': 0.20422383145424303, 'reward_scaling': 0.0009289155299795652, 'lookback': 321}. Best is trial 0 with value: 1818342038.625.


...

begin_total_asset:1000000
end_total_asset:2241709.040151249
Sharpe:  0.904504254797469
Trial 6 took 68.69712710380554 seconds.
begin_total_asset:1000000
end_total_asset:13764921.878772695
Sharpe:  0.5842344092670914
begin_total_asset:1000000
end_total_asset:12994635.786142927
Sharpe:  0.573882173002908
begin_total_asset:1000000
end_total_asset:10933510.591858013
Sharpe:  0.53985995225729
begin_total_asset:1000000
end_total_asset:13882318.042196792
Sharpe:  0.5840389190051307
begin_total_asset:1000000
end_total_asset:12739167.759997016
Sharpe:  0.5680054613816035
begin_total_asset:1000000
end_total_asset:14793565.772183068
Sharpe:  0.5969098092751713
begin_total_asset:1000000
end_total_asset:12830529.084447764
Sharpe:  0.5687623785097287
begin_total_asset:1000000
end_total_asset:14128234.731515093
Sharpe:  0.5870216403687315
begin_total_asset:1000000
end_total_asset:13496403.519756295
Sharpe:  0.5764703575910272
begin_total_asset:1000000
end_total_asset:14620045.402786877
Sharpe:  0.591

[I 2023-07-27 18:29:22,976] Trial 7 finished with value: 1823693361.375 and parameters: {'learning_rate': 0.00010597560270189279, 'n_steps': 48, 'gamma': 0.9618582875974061, 'gae_lambda': 0.962308067103787, 'ent_coef': 0.004271509238047536, 'vf_coef': 0.45969030542931705, 'max_grad_norm': 0.20345591047548378, 'reward_scaling': 0.00037510941264201793, 'lookback': 592}. Best is trial 7 with value: 1823693361.375.


begin_total_asset:1000000
end_total_asset:2329086.2886097683
Sharpe:  0.9328043377451568
Trial 7 took 83.25100803375244 seconds.


[W 2023-07-27 18:29:25,677] Trial 8 failed with parameters: {'learning_rate': 0.000552555276003299, 'n_steps': 19, 'gamma': 0.9617067907576219, 'gae_lambda': 0.8437077242544415, 'ent_coef': 0.0060009432005505665, 'vf_coef': 0.33801590316511376, 'max_grad_norm': 0.19854879387893526, 'reward_scaling': 7.553918115628101e-05, 'lookback': 300} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/Users/floriankockler/Documents/GitHub.nosync/QuantumAI/QuantumAI/hyper_opt/hyperopt1.py", line 99, in objective
    model.learn(total_timesteps=100000)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/stable_baselines3/a2c/a2c.py", line 194, in learn
    return super().learn(
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/stable_b

KeyboardInterrupt: 